In [ ]:
# -*- coding: utf-8 -*-
# 필요한 라이브러리 임포트
import os
import re
import torch
from pathlib import Path
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
from transformers import AutoProcessor, AutoModelForVision2Seq
from IPython.display import display

# ==============================================================================
# [CONFIG] 설정 섹션
# ==============================================================================

# 기본 설정
# --- 입력 이미지 경로 ---
IMAGE_PATH = Path("sample data/김광무_132.jpg")

# --- 모델 설정 ---
MODEL_ID_OR_PATH = "Qwen/Qwen2.5-VL-7B-Instruct"
USE_4BIT = False  # 4bit 양자화 사용 여부 (메모리 절약)
MAX_NEW_TOKENS = 768  # 모델의 최대 생성 토큰 수

# --- 외부 프롬프트 파일 경로 (지정하지 않으면 아래 기본 프롬프트 사용) ---
OCR_PROMPT_FILE = None    # 예: "prompts/ocr_prompt.txt"
CLEAN_PROMPT_FILE = None  # 예: "prompts/clean_prompt.txt"
COI_PROMPT_FILE = None    # 예: "prompts/coi_prompt.txt"

# ==============================================================================
# [PROMPTS] 프롬프트 정의 섹션
# ==============================================================================

#  1차: 원문 추출(OCR) 프롬프트
DEFAULT_OCR_PROMPT = """[ROLE]
너는 '식품 라벨 전문 OCR 감사관'이다. 사진 속 글자를 **원문 그대로** 받아적는 것이 전부다.

[STRICT RULES]
- 글자/숫자/기호/띄어쓰기/줄바꿈/단위(%, g, mg, kcal, ℃ 등) **한 글자도 바꾸지 말 것**.
- 보이지 않거나 확실치 않으면 **절대 추정/보완/번역 금지**. 그런 경우 해당 글자만 (?)
- 표/아이콘/도트와 함께 적힌 텍스트도 모두 **텍스트로 풀어 적기**.
- 출력 앞뒤에 설명/라벨/머리말/꼬리말/코드블록 금지. **텍스트만** 출력.

[캡처 우선순위]
1) 제품명
2) 원재료명(괄호 속 세부 성분 포함)
3) '알레르기 유발 물질' 표기(아이콘/표/문장)
4) ***교차오염/혼입 관련 문구*** (예: 같은 제조시설/라인/설비/공정, 교차오염, 혼입 가능, 함유 가능, 
   “이 제품은 … 사용한 제품과 같은 제조 시설에서 제조하였습니다” 등 **문장 전체를 원문 그대로**)

[표 처리]
- 표/영양성분 등은 '항목: 값' 한 줄 형식으로 적되, **원문 순서 유지**, 숫자·단위·괄호 보존.
  예) 나트륨: 680mg(34%)

[금지]
- 요약/재표현/정규화/정렬 변경/대소문자 변경/맞춤법 교정/의미추가 금지.
- 'assistant:' 같은 라벨 출력 금지.

[SELF-CHECK (내부)]
- 아래 항목이 최종 출력에 **실제로 존재**하는지 스스로 점검하되, 결과에는 **아무것도 추가하지 말 것**:
  (제품명 / 원재료명 / 알레르기 표기 / 교차오염 문구 후보)

지금부터 사진 속 텍스트를 **있는 그대로 전부** 적어라.
"""

# 2차: 정보 정리(Clean) 프롬프트
DEFAULT_CLEAN_PROMPT = """[ROLE]
너는 '식품 라벨 정리 도우미'다. 아래 [OCR] 원문을 **가감 없이** 정리한다.

[출력 형식(마크다운 섹션 제목은 정확히 다음과 같이)]
# 제품명
# 원재료명
# 알레르기 유발물질(원재료)
# 알레르기 유발물질(주의사항)
# 교차오염/혼입 가능 문구(원문 인용)

[채집 규칙]
- **모든 출력은 원문 그대로**(철자/띄어쓰기/괄호/단위 보존). 정규화/치환/요약 금지.
- 섹션은 비워두지 말고, 해당 내용이 전혀 없으면 **'표기 없음'**만 한 줄로 적기.
- "알레르기 유발물질(원재료)":
  [규칙] '원재료명'에 기재된 모든 성분을 아래 [판별 기준] 목록과 **반드시 비교**하여, 목록에 해당하는 성분이나 그 파생어(예: 소맥분, 대두유, 난백 등)를 포함하는 **원재료 항목을 원문 그대로** 모두 나열한다.
  [중요] 이 작업은 별도의 알레르기 표시가 빛 반사 등으로 누락된 경우를 대비한 **필수 교차 검증**이므로 절대 생략해선 안 된다.
  [예시] '원재료명'에 '...정제소금, 토마토페이스트, ...'가 있다면, [판별 기준]의 '토마토'에 해당하므로 '토마토페이스트'를 목록에 포함시켜야 한다.
- "알레르기 유발물질(주의사항)":
  1) [OCR]의 '알레르기 유발 물질' 표기(아이콘/표 포함)에서 언급된 항목을 원문 그대로 줄바꿈으로 나열.
  2) '교차오염/혼입 관련 문구'에 등장한 항목이 있으면 그것도 **원문 그대로** 추가.
  3) **중복 허용**.

[판별 기준(참조용 목록: 기록은 항상 원문 그대로)]
알류(가금류만 해당한다), 우유, 메밀, 땅콩, 대두, 밀, 고등어, 게, 새우,
돼지고기, 복숭아, 토마토, 아황산류(이를 첨가하여 최종 제품에 이산화황이 1킬로그램당 10밀리그램이상 함유된 경우만 해당한다),
호두, 닭고기, 쇠고기, 오징어, 조개류(굴, 전복, 홍합을포함한다), 잣

[교차오염/혼입 문구 섹션]
- [OCR]에서 해당 문장이 보이면 **문장 전체를 원문 그대로** 줄바꿈으로 적기.
- 없으면 **'표기 없음'**.

[금지]
- 의미추가/추정/통합/정리용 괄호/머리말/꼬리말/불릿 추가 금지.
- 라벨(예: 'assistant:') 출력 금지.

[OCR 원문]
{ocr_text}
"""

# 3차: 교차오염(COI) 문구 집중 추출 프롬프트
DEFAULT_COI_PROMPT = """[ROLE]
너는 '교차오염/혼입 문구 전용 추출기'다.

[목표]
사진에서 **교차오염/혼입 관련 문장만** 원문 그대로 복사해 출력한다.

[탐지 키워드/패턴(하나라도 포함되면 해당 '문장 전체'를 복사)]
- 키워드: 같은 제조시설, 같은 제조 시설, 같은 제조라인, 같은 제조 라인, 같은 설비, 같은 공정,
         교차오염, 교차 오염, 혼입 가능, 함유 가능
- 특수 문장 패턴(예시, …는 임의 텍스트):
  "이 제품은 … 사용한 제품과 같은 제조 시설에서 제조하였습니다"
  "이 제품은 … 사용한 제품과 같은 제조시설에서 제조하였습니다"
  "… 제품과 같은 제조 시설에서 제조"
- 한글/영문/숫자/괄호/기호/문장부호/띄어쓰기/‘…’ 포함 **그대로** 유지.

[출력 규칙]
- **각 문장을 한 줄에 하나씩** 출력. 다른 말 절대 금지.
- **글자 하나도 바꾸지 말 것**(띄어쓰기/마침표/괄호/기호/‘…’ 포함).
- 해당 문장이 전혀 없으면 **'없음'**만 출력.

[금지]
- 요약/재표현/추정/다른 문장과 병합/분할 금지.
- 어떤 라벨/설명/헤더/코드블록도 금지.

[SELF-CHECK (내부)]
- '같은 제조' / '교차오염' / '(혼입|함유) 가능' / '이 제품은 … 같은 제조 시설' 패턴이 있는지
  내부적으로 재확인하되, 결과에는 **아무것도 추가하지 말 것**.

지금부터 해당되는 문장만 출력하라.
"""

# ==============================================================================
# [HELPER FUNCTIONS] 보조 함수 섹션
# ==============================================================================

def load_prompt_from_file(path: str, default_text: str) -> str:
    """파일 경로가 유효하면 파일 내용을, 아니면 기본 텍스트를 반환합니다."""
    if path and Path(path).exists():
        return Path(path).read_text(encoding="utf-8")
    return default_text

def load_model_and_processor(model_id_or_path: str, use_4bit: bool):
    """지정된 모델과 프로세서를 로드합니다."""
    print(f" 모델 로딩 시작: {model_id_or_path}")
    if torch.cuda.is_available():
        dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
        device_map = "auto"
    else:
        dtype = torch.float32
        device_map = {"": "cpu"}

    processor = AutoProcessor.from_pretrained(model_id_or_path, trust_remote_code=True)
    model_kwargs = {"trust_remote_code": True}

    if use_4bit:
        model_kwargs.update({"load_in_4bit": True, "device_map": device_map})
    else:
        model_kwargs.update({"torch_dtype": dtype, "device_map": device_map})

    model = AutoModelForVision2Seq.from_pretrained(model_id_or_path, **model_kwargs)
    model.eval()
    print(" 모델 로딩 완료!")
    return model, processor

def generate_response(model, processor, image: Image.Image, prompt: str, max_new_tokens: int):
    """이미지와 프롬프트를 사용하여 모델의 응답을 생성합니다."""
    messages = [{"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": prompt}]}]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items() if hasattr(v, "to")}

    with torch.inference_mode():
        output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    
    decoded_output = processor.batch_decode(output_ids, skip_special_tokens=True)[0]
    # 어시스턴트 응답만 깔끔하게 추출
    split_key = "assistant"
    if split_key in decoded_output.lower():
        return decoded_output.split(split_key)[-1].strip(" :\n")
    return decoded_output.strip()

# --- 이미지 처리 및 COI 텍스트 분석 유틸리티 ---

def enhance_for_text(img: Image.Image) -> Image.Image:
    """OCR 성능 향상을 위해 이미지를 선명하게 만듭니다."""
    img = ImageOps.exif_transpose(img)
    img = img.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))
    img = ImageEnhance.Contrast(img).enhance(1.25)
    img = ImageEnhance.Sharpness(img).enhance(1.1)
    return img

def tile_image(img: Image.Image, overlap=0.12, zooms=(1.0, 1.25, 1.5)) -> list:
    """이미지를 여러 개의 타일로 잘라 리스트로 반환합니다."""
    W, H = img.size
    tiles = []
    for z in zooms:
        w, h = int(W / z), int(H / z)
        x_step = int(w * (1 - overlap))
        y_step = int(h * (1 - overlap))
        for y in range(0, H - h + 1, max(1, y_step)):
            for x in range(0, W - w + 1, max(1, x_step)):
                crop = img.crop((x, y, x + w, y + h))
                tiles.append(enhance_for_text(crop))
    return tiles

def find_coi_lines(text: str) -> list:
    """텍스트에서 교차오염 관련 문구(COI)를 찾아 리스트로 반환합니다."""
    COI_KEYWORDS = [
        "같은 제조시설", "같은 제조 시설", "같은 제조라인", "같은 제조 라인",
        "같은 설비", "같은 공정", "교차오염", "교차 오염", "혼입 가능", "함유 가능",
        "사용한 제품과 같은 제조 시설", "사용한 제품과 같은 제조시설",
    ]
    COI_PATTERNS = [
        r"같은\s*제조\s*시설", r"같은\s*제조\s*라인", r"같은\s*(설비|공정)",
        r"교차\s*오염", r"(혼입|함유)\s*가능",
        r"이\s*제품은.*사용한\s*제품과\s*같은\s*제조\s*시설(?:에서)?\s*제조(?:되었|하였)습니다[.\s]*",
    ]
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    hits = []
    for line in lines:
        if any(keyword in line for keyword in COI_KEYWORDS) or any(re.search(pattern, line) for pattern in COI_PATTERNS):
            hits.append(line)
    return sorted(set(hits), key=lambda t: (-len(t), t))

# ==============================================================================
# [WORKFLOW FUNCTIONS] 실행 흐름 함수 섹션
# ==============================================================================

def run_ocr(model, processor, image: Image.Image, output_path: Path):
    """1단계: 이미지에서 전체 텍스트를 추출(OCR)하고 파일로 저장합니다."""
    print("\n---  1단계: 전체 텍스트 추출 (OCR) 시작 ---")
    ocr_prompt = load_prompt_from_file(OCR_PROMPT_FILE, DEFAULT_OCR_PROMPT)
    ocr_text = generate_response(model, processor, image, ocr_prompt, MAX_NEW_TOKENS)
    
    output_path.write_text(ocr_text, encoding="utf-8")
    print(f" OCR 결과 저장 완료: {output_path}")
    print("-------------------------------------------\n")
    print(ocr_text[:500] + "...") # 미리보기
    print("\n-------------------------------------------")

def run_coi_extraction(model, processor, image: Image.Image, output_path: Path):
    """2단계: 교차오염(COI) 문구를 집중적으로 탐색하고 파일로 저장합니다."""
    print("\n---  2단계: 교차오염 문구 집중 탐색 시작 ---")
    coi_prompt = load_prompt_from_file(COI_PROMPT_FILE, DEFAULT_COI_PROMPT)
    coi_hits = set()

    # 1. 전체 이미지에서 탐색
    full_img_text = generate_response(model, processor, image, coi_prompt, 256)
    coi_hits.update(find_coi_lines(full_img_text))
    
    # 2. 이미지를 타일로 잘라 탐색
    tiles = tile_image(image)
    for i, tile in enumerate(tiles):
        print(f"  - 타일 이미지 분석 중 ({i+1}/{len(tiles)})...", end="\r")
        tile_text = generate_response(model, processor, tile, coi_prompt, 192)
        coi_hits.update(find_coi_lines(tile_text))
    
    # 3. 결과 정리 및 저장
    final_coi_text = "\n".join(sorted(list(coi_hits))) if coi_hits else "탐지된 교차오염 문구 없음"
    output_path.write_text(final_coi_text, encoding="utf-8")
    print(f"\n 교차오염 탐색 결과 저장 완료: {output_path}")
    print("-------------------------------------------\n")
    print(final_coi_text)
    print("\n-------------------------------------------")


def run_final_clean(model, processor, image: Image.Image, ocr_path: Path, coi_path: Path, output_path: Path):
    """3단계: OCR과 COI 결과를 통합하여 최종 정리된 텍스트를 생성하고 저장합니다."""
    print("\n---  3단계: 최종 정보 정리 및 요약 시작 ---")
    if not ocr_path.exists():
        print(f" OCR 파일({ocr_path})이 없어 최종 정리를 건너뜁니다.")
        return

    # OCR 결과와 추가 탐지된 COI 결과를 합침
    ocr_text = ocr_path.read_text(encoding="utf-8")
    coi_text = coi_path.read_text(encoding="utf-8") if coi_path.exists() else ""
    
    augmented_ocr_text = ocr_text
    if "탐지된 교차오염 문구 없음" not in coi_text and coi_text.strip():
        augmented_ocr_text += f"\n\n[추가 탐지된 교차오염 문구]\n{coi_text}"

    # 정리 프롬프트 준비 및 실행
    clean_prompt_template = load_prompt_from_file(CLEAN_PROMPT_FILE, DEFAULT_CLEAN_PROMPT)
    final_prompt = clean_prompt_template.format(ocr_text=augmented_ocr_text)
    
    clean_text = generate_response(model, processor, image, final_prompt, MAX_NEW_TOKENS)
    
    output_path.write_text(clean_text, encoding="utf-8")
    print(f" 최종 정리 결과 저장 완료: {output_path}")
    print("-------------------------------------------\n")
    print(clean_text)
    print("\n-------------------------------------------")

# ==============================================================================
# [MAIN] 메인 실행 블록
# ==============================================================================
if __name__ == "__main__":
    # 1. 파일 경로 설정
    if not IMAGE_PATH.exists():
        raise FileNotFoundError(f"이미지 파일을 찾을 수 없습니다: {IMAGE_PATH}")
    
    # 입력 파일명 기준으로 출력 파일 경로들 생성
    base_name = IMAGE_PATH.stem
    output_dir = IMAGE_PATH.parent
    raw_txt_path = output_dir / f"{base_name}_raw.txt"
    coi_txt_path = output_dir / f"{base_name}_coi.txt"
    clean_txt_path = output_dir / f"{base_name}_clean.txt"

    # 2. 모델 로드
    model, processor = load_model_and_processor(MODEL_ID_OR_PATH, USE_4BIT)
    
    # 3. 이미지 로드 및 미리보기
    print(f" 이미지 로드: {IMAGE_PATH}")
    image = Image.open(IMAGE_PATH).convert("RGB")
    display(ImageOps.contain(image, (600, 600)))

    # 4. 워크플로우 실행
    # 1단계: OCR 실행
    run_ocr(model, processor, image, raw_txt_path)
    
    # 2단계: 교차오염 문구 추출
    run_coi_extraction(model, processor, image, coi_txt_path)
    
    # 3단계: 최종 결과 정리
    run_final_clean(model, processor, image, raw_txt_path, coi_txt_path, clean_txt_path)

    print("\n 모든 작업이 완료되었습니다!")